<a href="https://colab.research.google.com/github/zhalok/Data-Science-Challenge/blob/main/OMR-Scanner-Grading-Using-Yolo-and-DBScan%20.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 608.9/608.9 kB 15.6 MB/s eta 0:00:00


In [2]:
from ultralytics import YOLO
import cv2
from google.colab.patches import cv2_imshow
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import DBSCAN
from sklearn.datasets import make_blobs
import matplotlib.pyplot as plt
import os

In [3]:
def euclidean_distance(point1, point2):
    return np.linalg.norm(point1 - point2)

In [4]:
def grayscale_conversion(image):
  gray_image = np.zeros_like(image)
  gray_image[:,:,0] = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
  gray_image[:,:,1] = gray_image[:,:,0]
  gray_image[:,:,2] = gray_image[:,:,0]

  return gray_image

In [5]:
def predict(model_path,image_path):
  model = YOLO(model_path)
  image = cv2.imread(image_path)
  # gray_image = grayscale_conversion(image)
  predictions = model.predict(save=True,show_labels=False,source=image)
  return predictions[0]

In [17]:
def clustering(centers,boxes,names,confs):

  X=centers
  eps = 1
  min_samples = 1
  dbscan = DBSCAN(eps=eps,min_samples=min_samples)
  dbscan.fit(X)

  labels = dbscan.labels_
  core_samples = dbscan.core_sample_indices_


  n_clusters = len(set(labels)) - (1 if -1 in labels else 0)


  clusters = [0]*n_clusters



  for idx, c in enumerate(centers):

    cls = int(boxes[idx].cls.tolist()[0])
    nm = names[cls]
    cnf = confs[idx]
    # print(type(cnf))
    if clusters[labels[idx]]==0:
      clusters[labels[idx]] = [c,nm,cnf]
    elif clusters[labels[idx]][2]<cnf:
      clusters[labels[idx]]  = [c,nm,cnf]

  return clusters


In [90]:
def column_detection(clusters,boxes,names):

  X = [cluster[0] for cluster in clusters]


  eps = 10
  min_samples = 1
  dbscan = DBSCAN(eps=eps,min_samples=min_samples)
  dbscan.fit((np.array(X)[:,0]).reshape(-1,1))

  labels = dbscan.labels_
  core_samples = dbscan.core_sample_indices_




  n_clusters = len(set(labels)) - (1 if -1 in labels else 0)

  # print("clusters",n_clusters)



  columns = [[] for i in range(n_clusters)]


  for idx, x in enumerate(X):
    columns[labels[idx]].append([x,clusters[idx][1]])

  return columns

In [103]:
def grading(predictions):
  boxes = predictions.boxes
  names = predictions.names
  confs = predictions.boxes.conf.tolist()
  centers=[]

  for box in boxes:

    xywh = box.xywh
    # cls = int(box.cls.tolist()[0])
    center = box.xywh[0][0:2]
    center = center.numpy().tolist()
    center = [round(cent) for cent in center]

    centers.append(center)


  centers = np.array(centers)

  clusters = clustering(centers,boxes,names,confs)
  columns = column_detection(clusters,boxes,names)

  # print("column len",len(columns))
  # print("column values",len(columns[0]))
  # print("column",columns[0])
  # for c in columns:
  #   print(len(c))
  for i in range(len(columns)):
    columns[i] = sorted(columns[i],key=lambda x:x[0][1])

  columns = sorted(columns,key = lambda x:x[0][0][0])
  # print(columns[0][0][0][0])

  answers = []
  for column in columns:
    # print(column)
    answers += column

  # for i in range(len(answers)):
  #   answers[i] = sorted(answers[i],key=lambda x:x[0][0])

  answers = [{"question":idx+1, "answer":c[1], "center":c[0]} for idx, c in enumerate(answers)]

  # answers

  return answers

In [37]:
def plot_answers(image_path,answers):
  font = cv2.FONT_HERSHEY_SIMPLEX
  font_scale = 2
  text_color = (255, 255, 255)  # Black color in BGR format
  text_thickness = 2
  image = cv2.imread(image_path)


  for idx,answer in enumerate(answers):
      cv2.putText(image, str(answer["answer"]).upper(), answer["center"], font, font_scale, text_color, text_thickness)

  cv2.imwrite("answer_plotting.jpg",image)
  cv2_imshow(image)

In [38]:
import gdown
file_url = "https://drive.google.com/uc?id=1GLY8NUGG_rEQPC0ctuf0v5UjyWciG5RS"
output_file = "model.pt"
gdown.download(file_url, output_file)

Downloading...
From: https://drive.google.com/uc?id=1GLY8NUGG_rEQPC0ctuf0v5UjyWciG5RS
To: /content/model.pt
100%|██████████| 6.24M/6.24M [00:00<00:00, 98.8MB/s]


'model.pt'

In [106]:
from google.colab import files
import json

uploaded = files.upload()

image_path = list(uploaded.keys())[0]
model_path = "model.pt"

predictions = predict(model_path=model_path,image_path=image_path)

os.remove(image_path)

answers = grading(predictions)
# print(answers)

# print(answers)

answers = [{"question_no":ans["question"],"answer":ans["answer"]} for ans in answers]


with open("answers.json", "w") as json_file:
    json.dump(answers, json_file,indent=4)
    print("Answers saved successfully")



Saving WhatsApp-Image-2023-06-13-at-11-25-19-PM-1-_jpeg.rf.26dcee4558839a5c9f14151cf891833b.jpg to WhatsApp-Image-2023-06-13-at-11-25-19-PM-1-_jpeg.rf.26dcee4558839a5c9f14151cf891833b.jpg



0: 640x480 7 as, 5 bs, 11 cs, 7 ds, 20 not answereds, 123.0ms
Speed: 2.8ms preprocess, 123.0ms inference, 0.8ms postprocess per image at shape (1, 3, 640, 480)
Results saved to runs/detect/predict2


Answers saved successfully


In [104]:
!rm -r runs